In [1]:
from model import ModelConfig, LlamaModel
from train import TrainerConfig, SimpleDataLoader, Trainer

from transformers import AutoTokenizer

In [2]:
tokenizer_id = "HuggingFaceTB/SmolLM2-135M"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
model_config = ModelConfig(
    vocab_size=tokenizer.vocab_size,
    d_model=576,
    d_head=64,
    d_mlp_proj=1536,
    n_layers=30,
    n_kv_heads=3,
    n_attn_heads=9,
    rms_norm_eps=1e-5,
    initializer_range=0.041666666666666664,
    rope_theta=100000.0,
    padding_idx=tokenizer.pad_token_id
)

In [5]:
train_config = TrainerConfig(
    per_device_train_batch_size=96,
    max_seq_len=192,
    num_epochs=64,
    eval_interval_steps=25,
    learning_rate=1e-4,
    grad_clip_norm=1.0,
    val_size=0.1,
    log_dir="runs/shakespeare",
    warmup_ratio=0.1
)

In [6]:
with open("data/tiny_shakespeare.txt") as f:
    text = f.read()

In [7]:
model = LlamaModel(model_config)
dataloader = SimpleDataLoader(train_config, tokenizer, text=text)
trainer = Trainer(train_config, model)

Total tokens                   | 341,184
Num Trainable Params           | 162,826,560
Train device                   | cuda, NVIDIA GeForce RTX 3090, N=1
Training precision             | torch.bfloat16
Flash Attention                | True
torch.compile()                | True
DistributedDataParallel        | False




In [8]:
trainer.train(dataloader)

Training steps                 | 544 
Step: 0, Training Loss: 11.35194, LR: 0.0000050, Tokens/sec: 1658.74
Step: 1, Training Loss: 11.32047, LR: 0.0000068, Tokens/sec: 1783.98
Step: 2, Training Loss: 11.26234, LR: 0.0000085, Tokens/sec: 87963.25
Step: 3, Training Loss: 11.22078, LR: 0.0000103, Tokens/sec: 87143.92
Computing Eval loss, steps: 2
Step: 3, Eval Loss: 11.19228
Step: 4, Training Loss: 11.19003, LR: 0.0000120, Tokens/sec: 90402.11
Step: 5, Training Loss: 11.13039, LR: 0.0000138, Tokens/sec: 87071.38
Step: 6, Training Loss: 11.06210, LR: 0.0000156, Tokens/sec: 95790.97
Step: 7, Training Loss: 10.95111, LR: 0.0000173, Tokens/sec: 94131.17
Step: 8, Training Loss: 10.80084, LR: 0.0000191, Tokens/sec: 93003.88
Step: 9, Training Loss: 10.70723, LR: 0.0000208, Tokens/sec: 94569.32
Step: 10, Training Loss: 10.50686, LR: 0.0000226, Tokens/sec: 95778.45
Step: 11, Training Loss: 10.41341, LR: 0.0000244, Tokens/sec: 91992.52
Step: 12, Training Loss: 10.22006, LR: 0.0000261, Tokens/sec: 7

In [9]:
input_text = """
All:
Content, content.

MENENIUS:
O sir, you are not right: have you not known
The worthiest men have done't?

CORIOLANUS:
""".strip()

input_ids = tokenizer([input_text], return_tensors="pt")['input_ids'].to(trainer.device)
idx = model.generate(input_ids, temperature=0.01, top_k=5, max_new_tokens=64)
print(tokenizer.batch_decode(idx)[0])

All:
Content, content.

MENENIUS:
O sir, you are not right: have you not known
The worthiest men have done't?

CORIOLANUS:
I am, no more.

MENENIUS:
A thousandly he dost!

CORIOLANUS:
I am, I have more by the people,
To see my good my master.

MENENIUS:
I am it more!
